## Reading data 

In [1]:
import sqlite3
import pandas as pd

In [2]:
db =sqlite3.connect('../Data/database.sqlite')
data= pd.read_sql_query(""" SELECT * from Reviews WHERE Score!=3 LIMIT 5000""",db) 

In [3]:
print(data.columns)
print(data.shape)

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')
(5000, 10)


In [4]:
score=data['Score']

In [5]:
classified_score=score.map(lambda x:int(x>3))
data.loc['Score'] = classified_score
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1.0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1.0,1.0,5.0,1.303862e+09,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2.0,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0.0,0.0,1.0,1.346976e+09,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3.0,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1.0,1.0,4.0,1.219018e+09,"""Delight"" says it all",This is a confection that has been around a fe...
3,4.0,B000UA0QIQ,A395BORC6FGVXV,Karl,3.0,3.0,2.0,1.307923e+09,Cough Medicine,If you are looking for the secret ingredient i...
4,5.0,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0.0,0.0,5.0,1.350778e+09,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
'''all the users who have given reviews more than 1 time'''
display0 = pd.read_sql_query("""                   
SELECT UserId, ProfileName, COUNT(*)
FROM Reviews
GROUP BY UserId
HAVING COUNT(*)>1
""", db)
display0.head()

,UserId,ProfileName,COUNT(*)
0,#oc-R115TNMSPFT9I7,Breyton,2
1,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",3
2,#oc-R11DNU2NBKQ23Z,Kim Cieszykowski,2
3,#oc-R11O5J5ZVQE25C,Penguin Chick,3
4,#oc-R12KPBODL2B5ZD,Christopher P. Presta,2


In [7]:
'''all the unique users '''
display1 = pd.read_sql_query("""                   
SELECT UserId, ProfileName, COUNT(*)
FROM Reviews
GROUP BY UserId
""", db)
display1.head()

,UserId,ProfileName,COUNT(*)
0,#oc-R103C0QSV1DF5E,C,1
1,#oc-R109MU5OBBZ59U,AayGee,1
2,#oc-R10LFEMQEW6QGZ,Julie,1
3,#oc-R10LT57ZGIB140,dipr,1
4,#oc-R10UA029WVWIUI,Kim D,1


In [8]:
print(str(display0.shape[0]) + " users who gave more than 1 reviews." )
print(str(display1.shape[0])+ " unique users out of " +  str(display1['COUNT(*)'].sum()) + " reviews.")

80668 users who gave more than 1 reviews.
256059 unique users out of 568454 reviews.


# Deduplication

In [9]:
'''duplicate reviews '''

display = pd.read_sql_query(""" 
SELECT UserId, ProfileName,Time,ProductId, COUNT(*)
FROM Reviews
GROUP BY UserId, Time 
HAVING COUNT(*)>2
""", db)
display.head()



,UserId,ProfileName,Time,ProductId,COUNT(*)
0,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ESG,3
1,#oc-R11O5J5ZVQE25C,Penguin Chick,1346889600,B005HG9ESG,3
2,#oc-R12MGTQS5KZZRV,"SKY2110 ""SKY2110""",1344211200,B005HG9ESG,3
3,#oc-R13NNUL4EKL4FL,N. Chernyavskaya,1348358400,B005HG9ESG,3
4,#oc-R14ZSRYW2YB41B,A. Crafton,1346284800,B005HG9ESG,3


In [10]:
display.iloc[0]['UserId']

'#oc-R11D9D7SHXIJB9'

In [11]:
duplicate = pd.read_sql_query(""" 
SELECT UserId, ProfileName,Time,ProductId,Text
FROM Reviews
WHERE UserID = "#oc-R11D9D7SHXIJB9"
""", db)
duplicate.head()

,UserId,ProfileName,Time,ProductId,Text
0,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ESG,"My wife has recurring extreme muscle spasms, u..."
1,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ERW,"My wife has recurring extreme muscle spasms, u..."
2,#oc-R11D9D7SHXIJB9,"Louis E. Emory ""hoppy""",1342396800,B005HG9ET0,"My wife has recurring extreme muscle spasms, u..."


Here, the same user reviews different product Id in same time which is not possible.  While examining the productIds B005HG9ESG, B005HG9ERW, B005HG9ET0 in Amazon, we found out that they referred to same product-Essentia Water.Hence, this is a data error. 

In [12]:
sorted_data=data.sort_values('ProductId', axis=0, ascending=True, inplace=False,ignore_index=True, kind='quicksort',na_position='last')
sorted_data.shape

(5001, 10)

In [13]:
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(4987, 10)

In [14]:
(final['Id'].size*1.0)/(data['Id'].size)*100

99.72005598880223

In [15]:
distorted_data = pd.read_sql_query(""" 
SELECT UserId, ProfileName,Time,ProductId,Text,HelpfulnessNumerator, HelpfulnessDenominator
FROM Reviews
WHERE HelpfulnessNumerator> HelpfulnessDenominator
""", db)
distorted_data

,UserId,ProfileName,Time,ProductId,Text,HelpfulnessNumerator,HelpfulnessDenominator
0,A2V0I904FH7ABY,Ram,1212883200,B001EQ55RW,It was almost a 'love at first bite' - the per...,3,2
1,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",1224892800,B000MIDROQ,My son loves spaghetti so I didn't hesitate or...,3,1


In [16]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [17]:
final.shape

(4986, 10)

## Text Preprocessing

In [18]:
import re

In [19]:
sample_0 = final['Text'].values[0]
print(sample_0)

sample_1= final['Text'].values[10]
print(sample_1)

sample_2 = final['Text'].values[11]
print(sample_2)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I bought this same brand from an online Indian grocery store that usually has excellent products.  I was able to turn it into cream/butter, using my super blender and adding some water, but it barely had any flavor.  I usually buy the Chao Kah brand of coconut cream (which is quite tasty and flavorful) and read another review for a different product for making your own coconut cream.  My complaint is not the shreds or texture.  Mine was just virtually tasteless.
I use these to keep my finicky toddler's protein levels up. What's great, is that they're delicious just about any way you can imagine cooking a sausage!<br /><br />A favorite Recipe: Sautee one large onion, about a tablespoon each dried basil and oregano, and some 

In [20]:
sample_0 = re.sub(r"http\S+", "", sample_0) #https://stackoverflow.com/a/40823105/4084039
sample_1 = re.sub(r"http\S+", "", sample_1) # remove urls from text python: 
sample_2 = re.sub(r"http\S+", "", sample_2) 


In [21]:
sample_0, sample_1, sample_2

('Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.',
 'I bought this same brand from an online Indian grocery store that usually has excellent products.  I was able to turn it into cream/butter, using my super blender and adding some water, but it barely had any flavor.  I usually buy the Chao Kah brand of coconut cream (which is quite tasty and flavorful) and read another review for a different product for making your own coconut cream.  My complaint is not the shreds or texture.  Mine was just virtually tasteless.',
 'I use these to keep my finicky toddler\'s protein levels up. What\'s great, is that they\'re delicious just about any way you can imagine cooking a sausage!<br /><br />A favorite Recipe: Sautee one large onion, about a tablespoon each dried basil and oregano, and some salt and pepper in about 2T of butter. Add sausage, tha

In [22]:
from bs4 import BeautifulSoup
# https://stackoverflow.com/questions/16206380/python-beautifulsoup-how-to-remove-all-tags-from-an-element
soup0 = BeautifulSoup(sample_0, 'lxml')


soup1 = BeautifulSoup(sample_1, 'lxml')


soup2 = BeautifulSoup(sample_2, 'lxml')
soup0.text, soup1.text, soup2.text


('Why is this $[...] when the same product is available for $[...] here? />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.',
 'I bought this same brand from an online Indian grocery store that usually has excellent products.  I was able to turn it into cream/butter, using my super blender and adding some water, but it barely had any flavor.  I usually buy the Chao Kah brand of coconut cream (which is quite tasty and flavorful) and read another review for a different product for making your own coconut cream.  My complaint is not the shreds or texture.  Mine was just virtually tasteless.',
 'I use these to keep my finicky toddler\'s protein levels up. What\'s great, is that they\'re delicious just about any way you can imagine cooking a sausage!A favorite Recipe: Sautee one large onion, about a tablespoon each dried basil and oregano, and some salt and pepper in about 2T of butter. Add sausage, that\'s been cut into coins

In [23]:
def decontracted(phrase):
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [24]:
sent_2 = decontracted(soup2.text)
print(sent_2)
print("="*50)

I use these to keep my finicky toddler is protein levels up. What is great, is that they are delicious just about any way you can imagine cooking a sausage!A favorite Recipe: Sautee one large onion, about a tablespoon each dried basil and oregano, and some salt and pepper in about 2T of butter. Add sausage, that is been cut into coins, and russet potatoes cut into 1/4" thick rounds. Cover all with chicken stock and simmer until potatoes are soft but not dissolved. Throw in a few handfuls of pre-washed baby spinach (I usually use about a half a pound, and frozen would work fine, just put it in a minute earlier). Stir until the spinach is cooked, and then serve! Enjoy!


In [25]:
sent_0 = re.sub("\S*\d\S*", "", soup0.text).strip()
print(sent_0)

Why is this $[...] when the same product is available for $[...] here? />The Victor  and  traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [26]:
sent_0 = re.sub('[^A-Za-z0-9]+', ' ', sent_0)
print(sent_0)

Why is this when the same product is available for here The Victor and traps are unreal of course total fly genocide Pretty stinky but only right nearby 


In [27]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/sims/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
from nltk.corpus import stopwords
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [29]:
# we are removing the words from the stop words list: 'no', 'nor', 'not'


stopwords= set(['the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [30]:
from tqdm import tqdm
preprocessed_reviews = []

for sentance in tqdm(final['Text'].values):
    sentance = re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    # https://gist.github.com/sebleier/554280
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(sentance.strip())

100%|██████████| 4986/4986 [00:02<00:00, 2373.28it/s]


In [31]:
preprocessed_reviews[15],preprocessed_reviews[100],preprocessed_reviews[1500]

('thank goodness mexgrocer love pico pica sauce cannot find product local grocery stores anymore internet shopping makes living podunk town bearable no long lists saved occasional trip big city first name basis ups fedex drivers usps shipments hit miss fit mailbox great not saddle head town pico pica favorite product good love',
 'although seems like great product definitely not minced cans say minced not ground beef cat likes minced not chunks not ground although eat ground cat food sometimes inviting delicious yet walks away hate packaging lies contents product shipping speedy value great long deliver promise stuck food cat wont eat',
 'wow far two two star reviews one obviously no idea ordering wants crispy cookies hey sorry reviews nobody good beyond reminding us look ordering chocolate oatmeal cookies not like combination not order type cookie find combo quite nice really oatmeal sort calms rich chocolate flavor gives cookie sort coconut type consistency let also remember tastes d

## Featurization

### Bag of words
- most primitive

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_reviews)
print( count_vect.get_feature_names()[:10])

['aa', 'aahhhs', 'aback', 'abandon', 'abates', 'abbott', 'abby', 'abdominal', 'abiding', 'ability']


In [33]:
print('No. of unique words in the corpus = ',len(count_vect.get_feature_names()) )

No. of unique words in the corpus =  12997


In [34]:
final_counts = count_vect.transform(preprocessed_reviews)
print('Transformed shape of the reviews ',final_counts.get_shape())

Transformed shape of the reviews  (4986, 12997)


### n-Grams

In [35]:
#ngram_range=(1,2) makes unigram and bigrams
#min_df=10, it ignore all the words whose document frequency is less than 10
#max_features=5000, build a vocabulary that only consider the top 5000 features based on document frequency
count_vect = CountVectorizer(ngram_range=(1,2), min_df=10, max_features=5000)
count_vect.fit(preprocessed_reviews)
print(count_vect.get_feature_names()[:10])
len(count_vect.get_feature_names())


['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']


3136

In [36]:
final_ngram_counts = count_vect.transform(preprocessed_reviews)

print('Transformed shape of the reviews ',final_ngram_counts.get_shape())
print("No. of unique words including both unigrams and bigrams ", final_ngram_counts.get_shape()[1])

Transformed shape of the reviews  (4986, 3136)
No. of unique words including both unigrams and bigrams  3136


### TF-IDF

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df=10)
tf_idf_vect.fit(preprocessed_reviews)
print("some unique words in the corpus",tf_idf_vect.get_feature_names()[0:10])
print("No. of unique words ",len(tf_idf_vect.get_feature_names()) )

some unique words in the corpus ['ability', 'able', 'able find', 'able get', 'absolute', 'absolutely', 'absolutely delicious', 'absolutely love', 'absolutely no', 'according']
No. of unique words  3136


### Word2Vec
- best for featurization of words 
- deep learning based technique, state of art

In [39]:
i=0
list_of_sentence=[]
for sentence in preprocessed_reviews:
    list_of_sentence.append(sentence.split())

In [42]:
from gensim.models import Word2Vec

#training from the data corpse
w2v_model=Word2Vec(list_of_sentence,min_count=5,size=50, workers=4)
print(w2v_model.wv.most_similar('great'))
print(w2v_model.wv.most_similar('worst'))

[('calorie', 0.9929611086845398), ('think', 0.9929596781730652), ('alternative', 0.9929579496383667), ('kids', 0.9929488301277161), ('snack', 0.9929308295249939), ('excellent', 0.9928534030914307), ('regular', 0.9924934506416321), ('tasty', 0.9924482107162476), ('care', 0.9924431443214417), ('especially', 0.992281436920166)]
[('types', 0.9994375705718994), ('varieties', 0.9993651509284973), ('turned', 0.9993372559547424), ('hands', 0.9993362426757812), ('style', 0.9993293285369873), ('results', 0.9993227124214172), ('agree', 0.999312698841095), ('must', 0.9993077516555786), ('part', 0.9993048906326294), ('miss', 0.9992778301239014)]


In [54]:
w2v_words = list(w2v_model.wv.vocab)
len(w2v_words),w2v_words[:10]

(3817,
 ['product',
  'available',
  'course',
  'total',
  'pretty',
  'stinky',
  'right',
  'nearby',
  'used',
  'ca'])

In [53]:
w2v_model.wv['play']  #gives word embedding if the word is present in the vocabulary
                      # doesn't gove result for the  word ='hero'

array([ 0.5203094 , -0.11524472, -0.03408759, -0.38975224, -0.17141978,
        0.22379215,  0.0444588 , -0.31028974, -0.3244121 ,  0.27165738,
       -0.15449592, -0.2604714 , -0.07527973,  0.24295427, -0.3318368 ,
       -0.27959922, -0.09513819, -0.04810994, -0.17595536,  0.17410964,
       -0.1983768 ,  0.15539403,  0.00426711, -0.22382073, -0.43328646,
       -0.02655589,  0.36367628, -0.14617625,  0.28331026,  0.24129988,
        0.35226297,  0.16753016,  0.03206442, -0.10143079,  0.04948033,
        0.03265184,  0.25808403,  0.4213499 , -0.04385334,  0.13915081,
       -0.33932033,  0.26552796,  0.01106461,  0.02015172,  0.0555771 ,
        0.0767256 , -0.13796654, -0.10924177,  0.00961755,  0.21975385],
      dtype=float32)